# Prestressed Beam Problem

## Input data

In [1]:
using JuMP, Ipopt, GLPK

In [2]:
# Geometry
L, b, h = 12., 0.1, 1. # m

# Load and material
q0 = 20.  # kN/m
E = 30e+6 # kPa

# Bounds
σmin, σmax = -15e+3, 1e+3  # kPa
δmin, δmax = -L/300, L/300 # m
f0max = 5e+3 # kN
cover = 0.05 # m
emax = (h-2*cover)/2;

# Beam maximum stress (top/bottom) and deflection
σt(f0, e) = -f0 / (b * h) - 6 / (b * h^2) * (q0 / 8 * L^2 - f0 * e)
σb(f0, e) = -f0 / (b * h) + 6 / (b * h^2) * (q0 / 8 * L^2 - f0 * e)
 δ(f0, e) =  5 / (32E * b * h^3) * L^4 * (q0 - 8 / L / L * f0 * e);

## Solving as a conventional problem

In [3]:
model = Model(Ipopt.Optimizer)
set_optimizer_attribute(model, "print_level", 0)

@variable(model, f0 >= 0)
@variable(model, 0 <= e <= emax)

@objective(model, Min, f0)

@constraint(model, σmin <= σb(f0, e) <= σmax)
@constraint(model, σmin <= σt(f0, e) <= σmax)
@constraint(model, δmin <=  δ(f0, e) <= δmax)

println(model)
optimize!(model)

println("Termination status: ", termination_status(model))
println("Primal status: ", primal_status(model))

println("f* : ", objective_value(model))
println("f0* : ", value(f0), ", e* : ", value(e))

Min f0
Subject to
 -60 f0*e - 10 f0 in [-36600.0, -20600.0]
 60 f0*e - 10 f0 in [6600.0, 22600.0]
 -5.9999999999999995e-5 f0*e in [-0.0616, 0.0184]
 f0 >= 0.0
 e >= 0.0
 e <= 0.45


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Termination status: LOCALLY_SOLVED
Primal status: FEASIBLE_POINT
f* : 556.7567421657133
f0* : 556.7567421657133, e* : 0.45


## Solving as a linear program
using $m = f0 \cdot e$

In [4]:
model = Model(GLPK.Optimizer)

@variable(model, f0 >= 0)
@variable(model, m >= 0)  # m = f0 * e

@objective(model, Min, f0)

# Beam maximum stress (top/bottom) and deflection
σt(f0, m) = -f0 / (b * h) - 6 / (b * h^2) * (q0 / 8 * L^2 - m)
σb(f0, m) = -f0 / (b * h) + 6 / (b * h^2) * (q0 / 8 * L^2 - m)
 δ(f0, m) =  5 / (32E * b * h^3) * L^4 * (q0 - 8 / L / L * m);

@constraint(model, σmin <= σb(f0, m) <= σmax)
@constraint(model, σmin <= σt(f0, m) <= σmax)
@constraint(model, δmin <=  δ(f0, m) <= δmax)

# Additional constraint
@constraint(model, m <= f0 * emax)

println(model)
optimize!(model)

println("Termination status: ", termination_status(model))
println("Primal status: ", primal_status(model))

println(" f* : ", objective_value(model))
println("f0* : ", value(f0), ", m* : ", value(m))
println(" e* : ", value(m)/value(f0))

Min f0
Subject to
 -0.45 f0 + m <= 0.0
 -10 f0 - 60 m in [-36600.0, -20600.0]
 -10 f0 + 60 m in [6600.0, 22600.0]
 -5.9999999999999995e-5 m in [-0.0616, 0.0184]
 f0 >= 0.0
 m >= 0.0

Termination status: OPTIMAL
Primal status: FEASIBLE_POINT
 f* : 556.7567567567568
f0* : 556.7567567567568, m* : 250.54054054054055
 e* : 0.45
